In [ ]:
from local.imports import *
from local.core import *

# Making Delegation in Python Work

- Author: Jeremy Howard
- Date: 2018-08-05

## The Delegation Problem

Let's look at a problem that all coders have faced; something that I call *the delegation problem*. To explain, I'll use an example. Here's an example class you might see in a content management system:

In [ ]:
class WebPage:
    def __init__(self, title, category="General", date=None, author="Jeremy"):
        self.title,self.category,self.author = title,category,author
        self.date = date or datetime.now()
        ...

Then, you want to add a subclass for certain types of page, such as a product page. It should have all the details of `WebPage`, plus some extra stuff. One way to do it would be with inheritance, like this:

In [ ]:
class ProductPage(WebPage):
    def __init__(self, title, sale_price, our_cost, category="General", date=None, author="Jeremy"):
        super().__init__(title, category=category, date=date, author=author)
        ...

But now we're violating the [Don't Repeat Yourself](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) (DRY) principal. We've duplicated both our list of parameter names, and the defaults. So later on, we might decide to change the default author to "Rachel", so we change the definition in `WebPage.__init__`. But we forget to do the same in `ProductPage`, and now we have a bug! When writing the [fastai](https://docs.fast.ai/) library I've created bugs many times in this way, and sometimes they've been extremely hard to track down, because differences in deep learning hyper-parameters can have very subtle and hard to test or detect implications.

To avoid this, perhaps we could instead write it this way:

In [ ]:
class ProductPage(WebPage):
    def __init__(self, title, sale_price, our_cost, **kwargs):
        super().__init__(title, **kwargs)
        ...

The key to this approach is the use of `**kwargs`. In python `**kwargs` in a parameter like means "put any additional keyword arguments into a dict called `kwarg`. And `**kwargs` in an argument list means "insert all key/value pairs in the `kwargs` dict as named arguments here". This approach is used in many popular libraries, such as `matplotlib`, in which the main `plot` function simply has the signature `plot(*args, **kwargs)`. The [plot documentation](https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot) says "*The `kwargs` are Line2D properties*" and then lists those properties.

It's not just python that uses this approach. For instance, in the [R language](https://www.r-project.org/) the equivalent to `**args` is simply written `...` (an *ellipsis*). The R documentation [explains](https://cran.r-project.org/doc/manuals/R-intro.html#The-three-dots-argument): "*Another frequent requirement is to allow one function to pass on argument settings to another. For example many graphics functions use the function par() and functions like plot() allow the user to pass on graphical parameters to par() to control the graphical output.This can be done by including an extra argument, literally ‘…’, of the function, which may then be passed on*".

For more details on using `**kwargs` in python, Google will find you many nice tutorials, such as [this one](https://www.digitalocean.com/community/tutorials/how-to-use-args-and-kwargs-in-python-3). 

The `**kwargs` solution appears to work quite nicely:

In [ ]:
p = ProductPage('Soap', 15.0, 10.50, category='Bathroom', author="Sylvain")
print(p.author)

Sylvain


However, this makes our API quite difficult to work with, because now the environment we're using for editing our Python code (examples in this article assume we're using Jupyter Notebook) doesn't know what parameters are available, so things like tab-completion of parameter names popup lists of signatures won't work. In addition, if we're using an automatic tool for generating API documentation (such as fastai's [show_doc](https://docs.fast.ai/gen_doc.nbdoc.html#show_doc) or [Sphinx](https://www.sphinx-doc.org/en/master/), our docs won't include the full list of parameters, and we'll need to manually add information about these *delegated parameters* (i.e. `category`, `date`, and `author`, in this case). In fact, we've seen this already, in matplotlib's documentation for `plot`.

Another alternative is to avoid inheritance, and instead use composition, like so:

In [ ]:
class ProductPage:
    def __init__(self, page, sale_price, our_cost):
        self.page,self.sale_price,self.our_cost = page,sale_price,our_cost
        ...

In [ ]:
p = ProductPage(WebPage('Soap', category='Bathroom', author="Sylvain"), 15.0, 10.50)
p.page.author

'Sylvain'

This has a new problem, however, which is that the most basic attributes are now hidden underneath `p.page`, which is not a great experience for our class users (and the constructor is now rather clunky compared to our inheritance version).

## Solving the problem with *delegated inheritance*

The solution to this that I've recently come up with is to create a new decorator that is used like this:

In [ ]:
@delegates()
class ProductPage(WebPage):
    def __init__(self, title, sale_price, our_cost, **kwargs):
        super().__init__(title, **kwargs)
        ...

...which looks exactly like what we had before for our inheritance version with `kwargs`, but has this key difference:

In [ ]:
print(inspect.signature(ProductPage))

(title, sale_price, our_cost, category='General', date=None, author='Jeremy')


It turns out that this little decorator solves all of our problems; in Jupyter if I hit the standard "show parameters" key <kbd>Shift</kbd>-<kbd>Tab</kbd> while instantiating a `ProductPage`, I see the full list of parameters, including those from `WebPage`. And hitting <kbd>Tab</kbd> will show me a completion list including the `WebPage` parameters. In addition, documentation tools see the full, correct signature, including the `WebPage` parameter details.